In [1]:
import fe
import load_data
import nn_models
import lossPloter as lp

import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

import os
import math
import cv2
import random

image resize options

In [2]:
patch_size = 384
patch_name = "" if patch_size==256 else "_p_" + str(patch_size)

# resize the input image to smaller size so that the same patch size covers
# larger area, from which the model may learn more global features
resize_scale = 0.5
resize_name = "" if resize_scale==1 else "_rs_" + str(resize_scale)
print(resize_name)

def resize_and_pad(nd_array, scale, patch_size):
    '''
    resize and pad the input array, so that the resolution fits the need, and the image is large enough for at least one patch
    '''
    if scale == 1:
        return nd_array
    h = nd_array.shape[0]
    w = nd_array.shape[1]
    h2,w2 = round(h*scale), round(w*scale)
    
    nd_array = cv2.resize(nd_array, (w2,h2))
    
    pad_h = max(0, 1 + (patch_size - h2) // 2)
    pad_w = max(0, 1 + (patch_size - w2) // 2)
    
    if pad_h > 0 or pad_w > 0:
        print (nd_array.shape)
        nd_array = np.pad(nd_array, ((pad_h, pad_h), (pad_w, pad_w)), "edge")
        print ("\t pad to", nd_array.shape)
        
    return nd_array

_rs_0.5


load catchmant names

In [6]:
catchment_root = "C:/Users/guo zifeng/Documents/2019 spring/00_Flood/03_Universal/catchments"
seed = 1
propotion = 3
load_num = 90 # set to None to load all

random.seed(seed)
with open(catchment_root + "/catchments.txt", "r") as text_file:
    catchments = [s.strip() for s in text_file.readlines()]
    random.shuffle(catchments)

if load_num is not None:
    catchments = catchments[:load_num]

catchments_train = catchments[len(catchments)//propotion:]
catchments_test = catchments[:len(catchments)//propotion]

print(catchments_train,catchments_test)

['669', '550', '443', '781', '524', '682', '848', '453', '358', '812', '372', '344', '552', '667', '786', '678', '442', '744', '661', '555', '665', '454', '762', '510', '488', '468', '685', '390', '502', '688', '564', '814', '778', '467', '787', '312', '709', '313', '777', '551', '388', '483', '509', '813', '302', '670', '660', '315', '683', '373', '466', '580', '681', '676', '684', '425', '501', '359', '745', '441'] ['859', '556', '333', '366', '763', '671', '689', '464', '553', '447', '663', '462', '849', '367', '528', '691', '868', '721', '334', '724', '465', '549', '565', '557', '484', '677', '435', '767', '843', '764']


specify the features

In [4]:
# segnet_default-NH0GD1l3
fe_x = fe.Features([fe.NormalizedHeight([]), fe.GeographicByDilate(1, keep_channels=[0,1,4])])
#fe_x = fe.Features([fe.HeightToDepth(scale = 0.01), fe.GeographicByDilate(1, keep_channels=[0,1,4])])

# segnet_default-NH0GK1-1fGK9-81fGK27-729f
# fe_x = fe.Features([fe.NormalizedHeight([]), fe.GeographicByKernel(1, 1), fe.GeographicByKernel(9, 81), fe.GeographicByKernel(27, 729)])

# segnet_default-NH4
# fe_x = fe.Features([fe.NormalizedHeight([64, 256, 512, 1024])])

# segnet_default-NH0GD1l1GD9l1GD81l1
# fe_x = fe.Features([fe.NormalizedHeight([]), fe.GeographicByDilate(1, keep_channels=[1]), fe.GeographicByDilate(9, keep_channels=[1]),  fe.GeographicByDilate(81, keep_channels=[1])])

load all images into the memory

In [7]:
fe.DEBUG=False # dont show the debug message

nd_img_train = [np.concatenate([fe_x.features(resize_and_pad(np.load(os.path.join(catchment_root, c + "_dem.npy")), resize_scale, patch_size)),resize_and_pad(np.load(os.path.join(catchment_root, c + "_waterdepth.npy")),resize_scale, patch_size)[...,None]],axis=-1) for c in catchments_train]
nd_img_test = [np.concatenate([fe_x.features(resize_and_pad(np.load(os.path.join(catchment_root, c + "_dem.npy")),resize_scale, patch_size)),resize_and_pad(np.load(os.path.join(catchment_root, c + "_waterdepth.npy")),resize_scale, patch_size)[...,None]],axis=-1) for c in catchments_test]

(462, 312)
	 pad to (462, 386)
(462, 312)
	 pad to (462, 386)


generate patches from in-memory images

In [8]:
# (Ellipsis, 0) means if all channels is 0
patch_train = load_data.PatchFromImage(nd_img_train, patch_size, Ellipsis, 0, ratio=5, batch_size=8,augmentation=True)
patch_test = load_data.PatchFromImage(nd_img_test, patch_size, Ellipsis, 0, ratio=5, batch_size=8)

37800 patches in total
5411 patches in total


build the CNN model

In [9]:
nnmodel = nn_models.unet_sym_k7_2(fe_x.channels(), 1, img_size=patch_size)

kernel [7, 7] out_channel 32
	 Tensor("encode0-0:0", shape=(?, 384, 384, 32), dtype=float32)
	 Tensor("encode0-1:0", shape=(?, 384, 384, 32), dtype=float32)
	 Tensor("encode0-p:0", shape=(?, 192, 192, 32), dtype=float32)
kernel [7, 7] out_channel 64
	 Tensor("encode1-0:0", shape=(?, 192, 192, 64), dtype=float32)
	 Tensor("encode1-1:0", shape=(?, 192, 192, 64), dtype=float32)
	 Tensor("encode1-p:0", shape=(?, 96, 96, 64), dtype=float32)
kernel [7, 7] out_channel 128
	 Tensor("encode2-0:0", shape=(?, 96, 96, 128), dtype=float32)
	 Tensor("encode2-1:0", shape=(?, 96, 96, 128), dtype=float32)
	 Tensor("encode2-p:0", shape=(?, 48, 48, 128), dtype=float32)
kernel [7, 7] out_channel 128
	 Tensor("encode3-0:0", shape=(?, 48, 48, 128), dtype=float32)
	 Tensor("encode3-1:0", shape=(?, 48, 48, 128), dtype=float32)
	 Tensor("encode3-p:0", shape=(?, 24, 24, 128), dtype=float32)
	 Tensor("latent-0:0", shape=(?, 24, 24, 128), dtype=float32)
	 Tensor("latent-1:0", shape=(?, 24, 24, 128), dtype=float32

loss and training operations

In [10]:
LR = 0.00002 #2e-5

nnmodel.phs["y"] = tf.placeholder(tf.float32,(None,patch_size,patch_size),name="y")
nnmodel.ls["loss"] = tf.reduce_mean(tf.square(nnmodel.phs["y"] - nnmodel.ls["prediction"]),name="loss")
train_op = tf.train.AdamOptimizer(LR).minimize(nnmodel.ls["loss"])
loss_op = nnmodel.ls["loss"]

In [11]:
model_path = "trained_models"
model_name = nnmodel.name + "-" + fe_x.name() + resize_name + patch_name

test_folder = "test_folder_" + model_name

print("model", model_name, "will be tested in folder", test_folder)

# make the test folder
if not os.path.exists(test_folder):
    os.mkdir(test_folder)

model unet_sym_kernel7_2-NH0GD1l3_rs_0.5_p_384 will be tested in folder test_folder_unet_sym_kernel7_2-NH0GD1l3_rs_0.5_p_384


start session

In [12]:
sess = tf.Session()
tf.global_variables_initializer().run(session=sess)
saver = tf.train.Saver()

start training

In [13]:
last_epoch = 0
model_id = "/-" + str(last_epoch-1)

epoch = 40
save_freq = 10

phs = nnmodel.phs

# loss calculator
train_loss = lp.Epoch_Loss()
test_loss = lp.Epoch_Loss()

%matplotlib qt
ploter = lp.Loss_Ploter()

# recover the loss record
if last_epoch > 0:
    print("restoring")
    ploter.load_record(model_path, model_name)
    saver.restore(sess,os.path.join(model_path,model_name+model_id))

print("training")

for e in range(epoch):
    test_loss.clear()
    train_loss.clear()

    # 1. training
    while not patch_train.end_of_epoch():
        xy=patch_train.next_batch()

        _, loss_val = sess.run([train_op,loss_op], feed_dict={phs["x"]:xy[:,:,:,:-1], phs["y"]:xy[:,:,:,-1]})

        # record batch loss
        train_loss.put_batch(loss_val)
        ploter.put_batch_loss(loss_val)

        # display batch loss curve
        if (ploter.count % 6==5):
            ploter.plot()

    # 2. testing
    while not patch_test.end_of_epoch():
        xy=patch_test.next_batch()
        loss_val = sess.run(loss_op, feed_dict={phs["x"]:xy[:,:,:,:-1], phs["y"]:xy[:,:,:,-1]})
        test_loss.put_batch(loss_val)

    ploter.put_epoch_loss(train_loss.get_epoch_loss(), test_loss.get_epoch_loss())
    print("epoch",e + last_epoch, "train_loss",train_loss.get_epoch_loss(),"test_loss",test_loss.get_epoch_loss())
    ploter.plot()
    ploter.savefig("training_plot.png")

    if e % save_freq == save_freq-1:
        saver.save(sess, os.path.join(model_path, model_name+"/"),e + last_epoch)
        ploter.save_record(model_path, model_name)

ploter.close()

training
epoch 0 train_loss 0.037060531453776455 test_loss 0.027489068781739675
epoch 1 train_loss 0.03117931850932085 test_loss 0.027615815123881556
epoch 2 train_loss 0.028338932957717035 test_loss 0.03348210864415615
epoch 3 train_loss 0.024128134515180828 test_loss 0.02451544529221886
epoch 4 train_loss 0.02253015158022877 test_loss 0.025780414507629423
epoch 5 train_loss 0.02064817090112243 test_loss 0.024570174270379667
epoch 6 train_loss 0.018471673374808338 test_loss 0.023726280144466694
epoch 7 train_loss 0.017854937743518758 test_loss 0.034621294348141624
epoch 8 train_loss 0.01579797515823018 test_loss 0.029686150589010002
epoch 9 train_loss 0.014102338537852225 test_loss 0.02236686693332495
epoch 10 train_loss 0.012945228926122938 test_loss 0.024527753808706216
epoch 11 train_loss 0.012108647497888241 test_loss 0.021293939271476808
epoch 12 train_loss 0.010803708430797246 test_loss 0.020873469605792914
epoch 13 train_loss 0.009755730773174456 test_loss 0.021183421884483192


KeyboardInterrupt: 

testing

In [14]:
def savepng(filename, img_array):
    cv2.imwrite(filename,cv2.cvtColor(img_array,cv2.COLOR_RGBA2BGRA))

def render(nd_array, cmap, vmin=0, vmax = 1, mask_indice=None):
    nd_array = (cmap((nd_array - vmin) / (vmax - vmin))*255).astype(np.uint8)
    if mask_indice is not None:
        nd_array[mask_indice] *= 0
    return nd_array

def run_test_in_session(model, sess, dem_array, patch_size, batch_size):
    ls = model.ls
    phs = model.phs

    height, width, _ = dem_array.shape
    
    # necessary data for prediction
    sum_x = np.zeros((height,width), dtype=np.float32)
    n_x = np.zeros((height,width), dtype=np.float32)
    
    # generate testing patches
    patches=[]
    num_patches = 0

    # generate patch locations
    patches = [[min(h,height-patch_size-1),min(w,width - patch_size-1)] for h in range(0, height,patch_size//2) for w in range(0, width, patch_size//2)]
    # select those that contains catchment areas
    patches = [[h, w] for h, w in patches if np.any(dem_array[h:h+patch_size,w:w+patch_size,1]>0)]
    num_patches = len(patches)
   
    patches = np.array(patches,dtype=np.uint32)

    for i in range(0, num_patches, batch_size):
        start = i
        end = min(i+batch_size, num_patches)
        
        cur_batch = patches[start:end]
        x = np.array([dem_array[h:h+patch_size,w:w+patch_size] for h,w in cur_batch])
        
        result = sess.run(ls["prediction"], feed_dict={phs["x"]:x[:,:,:,:-1]})
        
        for j in range(len(cur_batch)):
            h,w = cur_batch[j]
            sum_x[h:h+patch_size,w:w+patch_size] += result[j]
            n_x[h:h+patch_size,w:w+patch_size] += 1
    
    mean_x = np.copy(sum_x)
    mean_x[n_x>0] /= n_x[n_x>0]
    
    return mean_x

def render_result(location, name, prediction, dem_array):
    mask_indice = np.all(dem_array[:,:] == 0, axis=2) # mask
    
    img = render(prediction, plt.cm.terrain, 0, 5, mask_indice)
    cv2.imwrite(os.path.join(location,name+"_prediction.png"),cv2.cvtColor(img,cv2.COLOR_RGBA2BGRA))
    
    img = render(dem_array[:,:,-1], plt.cm.terrain, 0, 5, mask_indice)
    cv2.imwrite(os.path.join(location,name+"_truth.png"),cv2.cvtColor(img,cv2.COLOR_RGBA2BGRA))
    
    img = render(dem_array[:,:,0], plt.cm.terrain, -2, 2, mask_indice)
    cv2.imwrite(os.path.join(location,name+"_dem.png"),cv2.cvtColor(img,cv2.COLOR_RGBA2BGRA))
    
    error = prediction - dem_array[:,:,-1]
    
    img = render(error, plt.cm.seismic, -3, 3, mask_indice)
    cv2.imwrite(os.path.join(location,name+"_error.png"),cv2.cvtColor(img,cv2.COLOR_RGBA2BGRA))
    
    error = error[np.logical_not(mask_indice)]
    plt.figure(figsize=(8, 6))
    
    plt.hist(error, bins=[0.3*(i-40/2) for i in range(41)], color = "gray", rwidth=0.85)
    
    plt.yscale("log")
    plt.ylim((1, len(error)))
    plt.xlabel('Prediction Error',fontsize=16)
    plt.ylabel('Frequency',fontsize=16)
    
    plt.savefig(os.path.join(location,name+"_hist.png"))
    plt.close()

In [16]:
last_epoch = 20
model_id = "/-" + str(last_epoch-1)

print("restoring")
saver.restore(sess,os.path.join(model_path,model_name+model_id))
# tf.global_variables_initializer().run(session=sess)

print("testing")

%matplotlib inline
for i in range(20):
    result = run_test_in_session(nnmodel, sess, nd_img_train[i] , patch_size, 64)
    render_result(test_folder, "train_" + str(catchments_train[i]), result, nd_img_train[i])

for i in range(len(catchments_test)):
    result = run_test_in_session(nnmodel, sess, nd_img_test[i] , patch_size, 64)
    render_result(test_folder,  "test_" + str(catchments_test[i]), result, nd_img_test[i])

restoring
INFO:tensorflow:Restoring parameters from trained_models\unet_sym_kernel7_2-NH0GD1l3_rs_0.5_p_384/-19
testing
